# Project Overview

This notebook implements an advanced benchmark of four distinct Vision Language Models (VLMs) on the Hateful Memes Challenge Dataset (HMCD). This version loads the dataset from local files, uses a balanced dataset sample, and evaluates models from four different families to capture classification.

In [49]:
import ollama
import pandas as pd
import os

## Step 1: Load Dataset from Local Files & Prepare Sample

In [50]:
DATASET_FOLDER = 'data' 
ANNOTATION_FILE = os.path.join(DATASET_FOLDER, 'dev.jsonl')
IMG_DIR = os.path.join(DATASET_FOLDER, 'img')

In [51]:
if not os.path.exists(ANNOTATION_FILE):
    print(f"Error'")
else:
    df = pd.read_json(ANNOTATION_FILE, lines=True)
    # Create the full path to each image file
    df['img_path'] = df['img'].apply(lambda x: os.path.join(IMG_DIR, x))
    print("Local dataset loaded successfully.")


Local dataset loaded successfully.


### Balanced Sampling Strategy

In [52]:
df['label_text'] = df['label'].map({0: 'non-hateful', 1: 'hateful'})

In [53]:
hateful_df = df[df['label'] == 1].sample(n=20, random_state=70)

In [54]:
non_hateful_df = df[df['label'] == 0].sample(n=20, random_state=70)

In [55]:
df_sample = pd.concat([hateful_df, non_hateful_df])
df_sample = df_sample.sample(frac=1, random_state=50).reset_index(drop=True)

In [56]:
df_sample.head()

,id,img,label,text,img_path,label_text
0,56980,img/56980.png,0,coffee isn't helping get the jumper cables,data/img/img/56980.png,non-hateful
1,28905,img/28905.png,1,"hey, i just met you and this is crazy but here...",data/img/img/28905.png,hateful
2,32415,img/32415.png,0,when finishing a race is your passion,data/img/img/32415.png,non-hateful
3,97305,img/97305.png,1,when your wife just died giving birth to your ...,data/img/img/97305.png,hateful
4,63921,img/63921.png,0,taking a photo with family,data/img/img/63921.png,non-hateful


In [57]:
print(f"Created a balanced test set with {len(df_sample)} samples.")
print("Label distribution:")
print(df_sample['label_text'].value_counts())

Created a balanced test set with 40 samples.
Label distribution:
label_text
non-hateful    20
hateful        20
Name: count, dtype: int64


## Step 2: Playing with the Prompt

In [58]:
def create_zero_shot_prompt():
    """Asks for a direct classification."""
    return (
    "You are a content moderation AI. Your task is to classify a meme as either 'hateful' or 'non-hateful'. "
    "Analyze both the image and the accompanying text.\n"
    "Respond in *exactly* this format (no explanation before or after):\n"
    "Classification: hateful or non-hateful\n"
    "Justification: <your short reason>\n"
    "Example:\n"
    "Classification: non-hateful\n"
    "Justification: The meme contains no offensive content and appears neutral.\n"
)


In [59]:
def create_few_shot_prompt():
    return (
        "You are a content moderation AI. Your task is to classify a meme as either **hateful** or **non-hateful**.\n"
        "Analyze both the **image** and any **text** it contains. Think step-by-step, then give your final answer.\n\n"

        "Please follow the **exact format** below:\n"
        "Classification: <hateful|non-hateful>\n"
        "Justification: <1–2 sentence reason based on the image and/or text>\n\n"

        "### EXAMPLES ###\n\n"

        "Example 1:\n"
        "Image: A meme with a cartoon showing people of different races holding hands with the text: 'We’re stronger together.'\n"
        "Classification: non-hateful\n"
        "Justification: The image promotes unity and shows no offensive or targeted content.\n\n"

        "Example 2:\n"
        "Image: A meme with a photo of a specific religion’s attire and a caption mocking it with slurs.\n"
        "Classification: hateful\n"
        "Justification: The image contains derogatory language targeting a religious group, making it hateful.\n\n"

        "Example 3:\n"
        "Image: A meme with a man slipping on a banana peel with the caption: 'My Monday mornings.'\n"
        "Classification: non-hateful\n"
        "Justification: The image is humorous and contains no targeted or harmful content.\n\n"

        "### NEW MEME ###\n"
        "Now analyze the following meme and provide your response in the same format.\n"
        "Respond only with:\n"
        "Classification: <hateful|non-hateful>\n"
        "Justification: <reason>\n"
    )


In [60]:
# prompt_template = create_zero_shot_prompt()
prompt_template = create_few_shot_prompt()

In [61]:
# def parse_response(response_text):
#     cleaned_text = response_text.lower().strip()

#     classification = 'error'
#     justification = '' ''## will modify this later. Adding for debugging

#     if 'justification:' in cleaned_text:
#         try:
#             parts = cleaned_text.split('justification:', 1)
#             classification_part = parts[0]
#             justification = parts[1].strip()
#         except IndexError:
#             classification_part = cleaned_text
#             justification = "Could not parse justification part."
#     else:
#         classification_part = cleaned_text

#     if 'non-hateful' in classification_part:
#         classification = 'non-hateful'
#     elif 'hateful' in classification_part:
#         classification = 'hateful'
        
#     return classification, justification

In [62]:
## Parse function v3
import re

def parse_response(response_text):
    """
    A robust function to extract classification and justification using regular expressions.
    """
    cleaned_text = response_text.lower().strip()

    classification = 'error'
    justification = ''

    classification_match = re.search(r"classification:\s*(hateful|non-hateful)", cleaned_text) 
    
    if classification_match:
        classification = classification_match.group(1).strip()
    else:
        if 'non-hateful' in cleaned_text:
            classification = 'non-hateful'
        elif 'hateful' in cleaned_text:
            classification = 'hateful'

    justification_match = re.search(r"justification:\s*(.*)", cleaned_text, re.DOTALL)
    if justification_match:
        justification = justification_match.group(1).strip()
    elif not justification and classification != 'error':
        justification = "Justification not provided in response."
    else:
        justification = cleaned_text # For 'error' cases, store the raw output for debugging

    return classification, justification



In [63]:
# def parse_response(response_text): v2
#     cleaned_text = response_text.lower().strip()

#     classification = 'error'
#     justification = ''

#     # Try to extract classification line
#     if 'classification:' in cleaned_text:
#         try:
#             classification_line = cleaned_text.split('classification:')[1].split('\n')[0].strip()
#             if 'non-hateful' in classification_line:
#                 classification = 'non-hateful'
#             elif 'hateful' in classification_line:
#                 classification = 'hateful'
#         except Exception:
#             classification = 'error'

#     # Fallback: try first word
#     # if classification == 'error':
#     #     if cleaned_text.startswith('non-hateful'):
#     #         classification = 'non-hateful'
#     #     elif cleaned_text.startswith('hateful'):
#     #         classification = 'hateful'

#     # Extract justification
#     if 'justification:' in cleaned_text:
#         try:
#             justification = cleaned_text.split('justification:')[1].strip()
#         except IndexError:
#             justification = 'Could not parse justification.'

#     return classification, justification


In [64]:
import google.generativeai as genai
from PIL import Image
import os

In [65]:
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
if GOOGLE_API_KEY:
    genai.configure(api_key=GOOGLE_API_KEY)

In [66]:
def classify_with_gemini(image_path, prompt):
    """Classifies a meme using the Gemini 1.5 Flash model."""
    if not GOOGLE_API_KEY:
        return 'error', "GOOGLE_API_KEY not configured."
    try:
        model = genai.GenerativeModel('gemini-1.5-flash-latest')
        img = Image.open(image_path)
        
        safety_settings = {
            'HARM_CATEGORY_HATE_SPEECH': 'BLOCK_NONE',
            'HARM_CATEGORY_HARASSMENT': 'BLOCK_NONE',
            'HARM_CATEGORY_DANGEROUS_CONTENT': 'BLOCK_NONE'
        }
        
        response = model.generate_content([prompt, img], safety_settings=safety_settings)
        return parse_response(response.text)
    except Exception as e:
        return 'error', f"Gemini API Error: {str(e)}"


In [67]:
# import base64

In [68]:
# def image_to_base64(image_path):
#     with open(image_path, "rb") as image_file:
#         return base64.b64encode(image_file.read()).decode('utf-8')

In [69]:
from tqdm import tqdm

In [70]:
models_to_test = {
    'gemini-2.5-flash': classify_with_gemini
}
results_data = {model: [] for model in models_to_test}

for model_name, classification_func in models_to_test.items():
    print(f"\n--- Benchmarking model: {model_name} ---")
    predictions = []
    justifications = []
    for index, row in tqdm(df_sample.iterrows(), total=len(df_sample), desc=f"Processing {model_name}"):
        pred, just = classification_func(row['img_path'], prompt_template)
        predictions.append(pred)
        justifications.append(just)

    df_sample[f'prediction_{model_name}'] = predictions
    df_sample[f'justification_{model_name}'] = justifications


print("\n--- Benchmark Complete! ---")
# display(df_sample[['id', 'label_text'] + [f'prediction_{model}' for model in models_to_test.keys()]].head())

display(df_sample.head())



--- Benchmarking model: gemini-2.5-flash ---


Processing gemini-2.5-flash: 100%|██████████| 40/40 [00:58<00:00,  1.45s/it]


--- Benchmark Complete! ---


,id,img,label,text,img_path,label_text,prediction_gemini-2.5-flash,justification_gemini-2.5-flash
0,56980,img/56980.png,0,coffee isn't helping get the jumper cables,data/img/img/56980.png,non-hateful,error,Gemini API Error: 429 You exceeded your curren...
1,28905,img/28905.png,1,"hey, i just met you and this is crazy but here...",data/img/img/28905.png,hateful,error,Gemini API Error: 429 You exceeded your curren...
2,32415,img/32415.png,0,when finishing a race is your passion,data/img/img/32415.png,non-hateful,error,Gemini API Error: 429 You exceeded your curren...
3,97305,img/97305.png,1,when your wife just died giving birth to your ...,data/img/img/97305.png,hateful,error,Gemini API Error: 429 You exceeded your curren...
4,63921,img/63921.png,0,taking a photo with family,data/img/img/63921.png,non-hateful,error,Gemini API Error: 429 You exceeded your curren...


In [71]:
ground_truth = df_sample['label_text']


In [72]:
from sklearn.metrics import classification_report

for model_name in models_to_test.keys():
    print(f"\n--- Evaluation Report for: {model_name} ---")
    model_predictions = df_sample[f'prediction_{model_name}']
    model_predictions = model_predictions.apply(lambda x: x[0] if isinstance(x, tuple) else x)

    report = classification_report(
        ground_truth, 
        model_predictions, 
        labels=['hateful', 'non-hateful'], 
        zero_division=0)
    print(report)


--- Evaluation Report for: gemini-2.5-flash ---
              precision    recall  f1-score   support

     hateful       0.00      0.00      0.00      20.0
 non-hateful       0.00      0.00      0.00      20.0

   micro avg       0.00      0.00      0.00      40.0
   macro avg       0.00      0.00      0.00      40.0
weighted avg       0.00      0.00      0.00      40.0



In [73]:
columns_to_keep = ['id', 'text', 'label_text']
for model_name in models_to_test.keys():
    columns_to_keep.append(f'prediction_{model_name}')
    columns_to_keep.append(f'justification_{model_name}')

In [74]:
full_results_df = df_sample[columns_to_keep]

In [75]:
full_results_df.to_csv('gemini few shot.csv', index=False)